<a href="https://colab.research.google.com/github/xxshenanigans/AIB_Project6/blob/main/project_model(re).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#학습자 문항 정오답 확률 예측 모델

In [ ]:
!python --version 

Python 3.7.12


In [ ]:
%tensorflow_version 1.x

import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
!pip install -q tensorflow-gpu==1.15.4 

     |████████████████████████████████| 411.0 MB 12 kB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


##데이터셋 불러오기

In [ ]:
import os
import csv
import numpy as np
from sklearn.utils import shuffle 

In [ ]:
# 문제 시퀀스의 최대 문제 개수에 따라 문제/정오답 시퀀스의 오른쪽에 target_value(-1)로 패딩 

def pad(data, target_length, target_value=0):
    return np.pad(data, (0, target_length - len(data)), 'constant', constant_values=target_value)

In [ ]:
# 학습을 위한 형태로 변환 

def one_hot(indices, depth):
    encoding = np.concatenate((np.eye(depth), [np.zeros(depth)]))
    indices[indices!=-1] -= 1
    return encoding[indices]

In [ ]:
class OriginalInputProcessor(object):
    def process_problems_and_corrects(self, problem_seqs, correct_seqs, num_problems, is_train=True):
        """
        This function aims to process the problem sequence and the correct sequence into a DKT feedable X and y.
        :param problem_seqs: it is in shape [batch_size, None]
        :param correct_seqs: it is the same shape as problem_seqs
        :return:
        """
        # pad the sequence with the maximum sequence length
        max_seq_length = max([len(problem) for problem in problem_seqs])
        problem_seqs_pad = np.array([pad(problem, max_seq_length, target_value=-1) for problem in problem_seqs])
        correct_seqs_pad = np.array([pad(correct, max_seq_length, target_value=-1) for correct in correct_seqs])

        # find the correct seqs matrix as the following way:
        # Let problem_seq = [1,3,2,-1,-1] as a and correct_seq = [1,0,1,-1,-1] as b, which are padded already
        # First, find the element-wise multiplication of a*b*b = [1,0,2,-1,-1]
        # Then, for any values 0, assign it to -1 in the vector = [1,-1,2,-1,-1] as c
        # Such that when we one hot encoding the vector c, it will results a zero vector
        temp = problem_seqs_pad * correct_seqs_pad * correct_seqs_pad  # temp is c in the comment.
        temp[temp == 0] = -1
        correct_seqs_pad = temp

        # one hot encode the information
        problem_seqs_oh = one_hot(problem_seqs_pad, depth=num_problems)
        correct_seqs_oh = one_hot(correct_seqs_pad, depth=num_problems)

        # slice out the x and y
        if is_train:
            x_problem_seqs = problem_seqs_oh[:, :-1]
            x_correct_seqs = correct_seqs_oh[:, :-1]
            y_problem_seqs = problem_seqs_oh[:, 1:]
            y_correct_seqs = correct_seqs_oh[:, 1:]
        else:
            x_problem_seqs = problem_seqs_oh[:, :]
            x_correct_seqs = correct_seqs_oh[:, :]
            y_problem_seqs = problem_seqs_oh[:, :]
            y_correct_seqs = correct_seqs_oh[:, :]

        X = np.concatenate((x_problem_seqs, x_correct_seqs), axis=2)

        result = (X, y_problem_seqs, y_correct_seqs)
        return result

In [ ]:
class BatchGenerator:
    """
    Generate batch for DKT model
    """

    def __init__(self, problem_seqs, correct_seqs, num_problems, batch_size, input_processor=OriginalInputProcessor(),
                 **kwargs):
        self.cursor = 0  # point to the current batch index
        self.problem_seqs = problem_seqs
        self.correct_seqs = correct_seqs
        self.batch_size = batch_size
        self.num_problems = num_problems
        self.num_samples = len(problem_seqs)
        self.num_batches = len(problem_seqs) // batch_size + 1 if len(problem_seqs) % batch_size!=0 else len(problem_seqs) // batch_size
        self.input_processor = input_processor
        self._current_batch = None

    def next_batch(self, is_train=True):
        start_idx = self.cursor * self.batch_size
        end_idx = min((self.cursor + 1) * self.batch_size, self.num_samples)
        problem_seqs = self.problem_seqs[start_idx:end_idx]
        correct_seqs = self.correct_seqs[start_idx:end_idx]

        # x_problem_seqs, x_correct_seqs, y_problem_seqs, y_correct_seqs
        self._current_batch = self.input_processor.process_problems_and_corrects(problem_seqs,
                                                                                 correct_seqs,
                                                                                 self.num_problems,
                                                                                 is_train=is_train)
        self._update_cursor()
        return self._current_batch

    @property
    def current_batch(self):
        if self._current_batch is None:
            print("Current batch is None.")
        return None

    def _update_cursor(self):
        self.cursor = (self.cursor + 1) % self.num_batches

    def reset_cursor(self):
        self.cursor = 0

    def shuffle(self):
        self.problem_seqs, self.correct_seqs = shuffle(self.problem_seqs, self.correct_seqs, random_state=42)

In [ ]:
def read_data_from_csv(filename):
    # read the csv file
    rows = []
    with open(filename, 'r') as f:
        print("Reading {0}".format(filename))
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            rows.append(row)
        print("{0} lines was read".format(len(rows)))

    # tuples stores the student answering sequence as
    # ([num_problems_answered], [problem_ids], [is_corrects])
    max_seq_length = 0
    # num_problems = 0
    num_problems = 1865
    
    tuples = []
    for i in range(0, len(rows), 3):
        # numbers of problem a student answered
        seq_length = len(rows[i + 1])

        # only keep student with at least 3 records.
        if seq_length < 3:
            continue

        problem_seq = rows[i + 1]
        correct_seq = rows[i + 2]

        invalid_ids_loc = [i for i, pid in enumerate(problem_seq) if pid == '']
        for invalid_loc in invalid_ids_loc:
            del problem_seq[invalid_loc]
            del correct_seq[invalid_loc]

        # convert the sequence from string to int.
        problem_seq = list(map(int, problem_seq))
        correct_seq = list(map(int, correct_seq))

        tup = (seq_length, problem_seq, correct_seq)
        tuples.append(tup)

        if max_seq_length < seq_length:
            max_seq_length = seq_length

        # pid = max(int(pid) for pid in problem_seq if pid != '')
        
        # if num_problems < pid:
        #     num_problems = pid

    print("max_num_problems_answered:", max_seq_length)
    print("num_problems:", num_problems)
    print("The number of data is {0}".format(len(tuples)))
    print("Finish reading data.")

    return tuples, num_problems, max_seq_length

In [ ]:
class DKTData:
    def __init__(self, train_path, valid_path, test_path, batch_size=32):
        self.students_train, num_problems_train, max_seq_length_train = read_data_from_csv(train_path)
        self.students_valid, num_problems_valid, max_seq_length_valid = read_data_from_csv(valid_path)
        self.students_test, num_problems_test, max_seq_length_test = read_data_from_csv(test_path)
        self.num_problems = max(num_problems_test, num_problems_train, num_problems_valid)
        self.max_seq_length = max(max_seq_length_train, max_seq_length_test, max_seq_length_valid)

        problem_seqs = [student[1] for student in self.students_train]
        correct_seqs = [student[2] for student in self.students_train]
        self.train = BatchGenerator(problem_seqs, correct_seqs, self.num_problems, batch_size)

        problem_seqs = [student[1] for student in self.students_valid]
        correct_seqs = [student[2] for student in self.students_valid]
        self.valid = BatchGenerator(problem_seqs, correct_seqs, self.num_problems, batch_size)
        
        problem_seqs = [student[1] for student in self.students_test]
        correct_seqs = [student[2] for student in self.students_test]
        self.test = BatchGenerator(problem_seqs, correct_seqs, self.num_problems, batch_size)

##모델 구축 

In [ ]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
    seq_length = tf.reduce_sum(used, 1)
    seq_length = tf.cast(seq_length, tf.int32)
    return seq_length

In [ ]:
# RNN, LSTM 모델 적용
# 손실 함수 생성 
# sigmoid에서 확률에 따라 target class를 0 또는 1로 분류 
# optimizer 설정 

class Model(object):
    def __init__(self, num_problems,
                 hidden_layer_structure=(200,),
                 batch_size=32,
                 rnn_cell=tf.contrib.rnn.LSTMCell,
                 learning_rate=0.01,
                 max_grad_norm=5.0,
                 lambda_w1 = 0.0,
                 lambda_w2 = 0.0,
                 lambda_o = 0.0,
                 **kwargs):
        self.num_problems = num_problems
        self.hidden_layer_structure = hidden_layer_structure
        self.batch_size = batch_size
        self.rnn_cell = rnn_cell
        self.learning_rate = learning_rate
        self.max_grad_norm = max_grad_norm
        self.lambda_w1 = lambda_w1
        self.lambda_w2 = lambda_w2
        self.lambda_o = lambda_o

    def _create_placeholder(self):
        print("Creating placeholder...")
        num_problems = self.num_problems
        self.X = tf.placeholder(tf.float32, [None, None, 2 * num_problems], name='X')
        self.y_seq = tf.placeholder(tf.float32, [None, None, num_problems], name='y_seq')
        self.y_corr = tf.placeholder(tf.float32, [None, None, num_problems], name='y_corr')
        self.keep_prob = tf.placeholder_with_default(1.0, shape=(), name='keep_prob')
        self.hidden_layer_input = self.X
        self.seq_length = length(self.X)

    def _influence(self):
        print("Creating Loss...")
        hidden_layer_structure = self.hidden_layer_structure
        self.hidden_layers_outputs = []
        self.hidden_layers_state = []
        hidden_layer_input = self.hidden_layer_input
        print("LSTM input shape: {0}".format(np.shape(hidden_layer_input)))
        for i, layer_state_size in enumerate(hidden_layer_structure):
            variable_scope_name = "hidden_layer_{}".format(i)
            with tf.variable_scope(variable_scope_name, reuse=tf.get_variable_scope().reuse):
                cell = self.rnn_cell(num_units=layer_state_size)
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)
                outputs, state = tf.nn.dynamic_rnn(
                    cell,
                    hidden_layer_input,
                    dtype=tf.float32,
                    sequence_length=self.seq_length
                )
            self.hidden_layers_outputs.append(outputs)
            self.hidden_layers_state.append(state)
            hidden_layer_input = outputs

    def _create_loss(self):
        print("Creating Loss...")
        last_layer_size = self.hidden_layer_structure[-1]
        last_layer_outputs = self.hidden_layers_outputs[-1]
        with tf.variable_scope("output_layer", reuse=tf.get_variable_scope().reuse):
            W_yh = tf.get_variable("weights", shape=[last_layer_size, self.num_problems],
                                   initializer=tf.random_normal_initializer(stddev=1.0 / np.sqrt(self.num_problems)))
            b_yh = tf.get_variable("biases", shape=[self.num_problems, ],
                                   initializer=tf.random_normal_initializer(stddev=1.0 / np.sqrt(self.num_problems)))
            num_steps = tf.shape(last_layer_outputs)[1]
            self.outputs_flat = tf.reshape(last_layer_outputs, shape=[-1, last_layer_size])
            self.logits_flat = tf.matmul(self.outputs_flat, W_yh) + b_yh
            self.logits = tf.reshape(self.logits_flat, shape=[-1, num_steps, self.num_problems])
            self.preds = tf.sigmoid(self.logits, name="preds")
            target_indices = tf.where(tf.not_equal(self.y_seq, 0))
            self.target_logits = tf.gather_nd(self.logits, target_indices)
            self.target_preds = tf.gather_nd(self.preds, target_indices)
            self.target_labels = tf.gather_nd(self.y_corr, target_indices)
            self.cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.target_logits,
                                                                         labels=self.target_labels)
            self.loss = tf.reduce_mean(self.cross_entropy)
            current_seq = self.X[:,:,:self.num_problems]
            current_corr = self.X[:,:,self.num_problems:]
            self.target_indices_current = tf.where(tf.not_equal(current_seq, 0))
            self.target_logits_current = tf.gather_nd(self.logits, self.target_indices_current)
            self.target_preds_current = tf.gather_nd(self.preds, self.target_indices_current) 
            self.target_labels_current = tf.gather_nd(current_corr, self.target_indices_current)
            self.cross_entropy_current = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.target_logits_current,
                                                                                 labels=self.target_labels_current)
            self.loss += self.lambda_o * tf.reduce_mean(self.cross_entropy_current)
            mask = length(self.y_seq)
            self.total_num_steps = tf.reduce_sum(tf.cast(mask, tf.float32))
            waviness_norm_l1 = tf.abs(self.preds[:, 1:, :] - self.preds[:, :-1, :])
            self.waviness_l1 = tf.reduce_sum(waviness_norm_l1) / self.total_num_steps / self.num_problems
            self.loss += self.lambda_w1 * self.waviness_l1
            waviness_norm_l2 = tf.square(self.preds[:, 1:, :] - self.preds[:, :-1, :])
            self.waviness_l2 = tf.reduce_sum(waviness_norm_l2) / self.total_num_steps / self.num_problems
            self.loss += self.lambda_w2 * self.waviness_l2

    def _create_optimizer(self):
        print('Create optimizer...')
        with tf.variable_scope('Optimizer'):
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            gvs = self.optimizer.compute_gradients(self.loss)
            clipped_gvs = [(tf.clip_by_norm(grad, self.max_grad_norm), var) for grad, var in gvs]
            self.train_op = self.optimizer.apply_gradients(clipped_gvs)

    def _add_summary(self):
        pass

    def build_graph(self):
        self._create_placeholder()
        self._influence()
        self._create_loss()
        self._create_optimizer()
        self._add_summary()
        tf.get_variable_scope().reuse_variables()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import sys, time, datetime, pytz
from sklearn.metrics import roc_curve, auc, accuracy_score
import pandas as pd
from tqdm import tqdm 

In [ ]:
SPLIT_MSG = "***********"

In [ ]:
def _seq_length(sequence):
    """
    This function return the sequence length of each x in the batch.
    :param sequence: the batch sequence of shape [batch_size, num_steps, feature_size]
    :return length: A tensor of shape [batch_size]
    """
    used = np.sign(np.max(np.abs(sequence), 2))
    seq_length = np.sum(used, 1)
    # seq_length = np.cast(seq_length, tf.int32)
    return seq_length

In [ ]:
# 학습 로그 생성
# 평가도구 acc_score/auc_score 활용, 평균값 출력 
# confusion_matrix 생성
# best result 저장, 10 epochs 동안 성능이 향상되지 않으면 학습 중지
# 문제/정오답 시퀀스에 대한 heatmap 출력  

class DKT(object):
    def __init__(self, sess, data_train, data_valid, data_test, num_problems, network_config, 
                 save_dir_prefix='/content/drive/MyDrive/Project_re/', num_runs=5, num_epochs=500, keep_prob=0.5, logging=True, 
                 save=True):
        self.sess = sess

        self.data_train = data_train
        self.data_valid = data_valid
        self.data_test = data_test
        self.num_problems = num_problems
        self.network_config = network_config
        self.model = Model(num_problems=num_problems, **network_config)
        self.keep_prob = keep_prob
        self.num_epochs = num_epochs
        self.num_runs = num_runs
        self.run_count = 0

        cell_type_str = repr(network_config['rnn_cell']).split('.')[-1][:-6]
        layer_structure_str = "-".join([str(i) for i in network_config['hidden_layer_structure']])
        model_name = self.model_name = cell_type_str + '-' + layer_structure_str
        save_dir_name = ""

        save_dir_name = 'n{}.lo{}.lw1{}.lw2{}'.format(layer_structure_str,
                                                    network_config['lambda_o'],
                                                    network_config['lambda_w1'],
                                                    network_config['lambda_w2'])
        save_dir_name += "/"

        self.ckpt_save_dir = os.path.join(save_dir_prefix, 'model')
        self.log_save_dir = save_dir_prefix
        # print('ckpt_save_dir: ', self.ckpt_save_dir)
        # print('log_save_dir: ', self.log_save_dir)

        if not os.path.exists(self.log_save_dir):
            os.makedirs(self.log_save_dir)
        self.log_file_path = os.path.join(self.log_save_dir, "model_train.log")
        self.logging = logging
        self.save = save

        self._log("Network Configuration:")
        for k, v in network_config.items():
            log_msg = "{}: {}".format(k, v)
            self._log(log_msg)
        self._log("Num of problems: {}".format(num_problems))
        self._log("Num of run: {}".format(num_runs))
        self._log("Max num of run: {}".format(num_epochs))
        self._log("Keep Prob: {}".format(keep_prob))

    def train(self):
        data = self.data_train
        model = self.model
        keep_prob = self.keep_prob
        sess = self.sess

        loss = 0.0
        y_pred = []
        y_true = []
        iteration = 1
        for batch_idx in tqdm(range(data.num_batches), desc="train batch..."):
            X_batch, y_seq_batch, y_corr_batch = data.next_batch()
            feed_dict = {
                model.X: X_batch,
                model.y_seq: y_seq_batch,
                model.y_corr: y_corr_batch,
                model.keep_prob: keep_prob,
            }
            _, _target_preds, _target_labels, _loss = sess.run(
                [model.train_op, model.target_preds, model.target_labels, model.loss],
                feed_dict=feed_dict
            )
            y_pred += [p for p in _target_preds]
            y_true += [t for t in _target_labels]
            loss = (iteration - 1) / iteration * loss + _loss / iteration
            iteration += 1
        try:
            acc_score = accuracy_score(np.array(y_true), np.round(y_pred))
            fpr, tpr, thres = roc_curve(y_true, y_pred, pos_label=1)
            auc_score = auc(fpr, tpr)
        except ValueError:
            self._log("Value Error is encountered during finding the acc_score and auc_score. Assign the AUC to 0 now.")
            acc_score = 0.0
            auc_score = 0.0
            loss = 999999.9

        return acc_score, auc_score, loss

    def evaluate(self, mode='valid', makefile=False):
        if mode == 'train':
            data = self.data_train
        elif mode == 'valid':
            data = self.data_valid
        else:
            data = self.data_test

        data.reset_cursor()
        model = self.model
        sess = self.sess
        
        total_confusion_matrix = list()
        y_pred = []
        y_true = []
        y_pred_current = []
        y_true_current = []
        iteration = 1
        loss = 0.0
        acc_score = 0.0
        auc_score_current = 0.0
        auc_score = 0.0
        for batch_idx in tqdm(range(data.num_batches), desc=f"{mode} batch..."):
            X_batch, y_seq_batch, y_corr_batch = data.next_batch()
            feed_dict = {
                model.X: X_batch,
                model.y_seq: y_seq_batch,
                model.y_corr: y_corr_batch,
                model.keep_prob: 1,
            }
            _target_preds, _target_labels, _target_preds_current, _target_labels_current, _loss = sess.run(
                [model.target_preds,
                 model.target_labels,
                 model.target_preds_current,
                 model.target_labels_current,
                 model.loss],
                feed_dict=feed_dict
            )
            
            y_pred += [_target_preds[p] for p in range(len(_target_preds))]
            y_true += [_target_labels[t] for t in range(len(_target_labels))]
            y_pred_current += [_target_preds_current[p]  for p in range(len(_target_preds_current))]
            y_true_current += [_target_labels_current[t]  for t in range(len(_target_labels_current))]
            loss = (iteration - 1) / iteration * loss + _loss / iteration
            iteration += 1

            confusion = list()
            
            if makefile:
                for i in range(len(_target_preds)):
                     _target_preds[i] = int(np.round(_target_preds[i]))
                     _target_labels[i] = int(_target_labels[i])
                     if _target_preds[i] == _target_labels[i]:
                         if _target_preds[i] == 0: # (true, pred) = (0,0) -> TN
                            confusion.append('TN')
                         else: #(true, pred) = (1, 1)
                            confusion.append('TP')
                     else:
                         if _target_preds[i] == 0: #(true, pred) = (1, 0) ->FN
                             confusion.append('FN')
                         else:
                             confusion.append('FP')
    
                total_confusion_matrix.append(confusion)
            
        try:
            acc_score = accuracy_score(np.array(y_true), np.round(y_pred))
            fpr, tpr, thres = roc_curve(y_true, y_pred, pos_label=1)
            auc_score = auc(fpr, tpr)
            fpr, tpr, thres = roc_curve(y_true_current, y_pred_current, pos_label=1)
            auc_score_current = auc(fpr, tpr)
        except ValueError:
            self._log("Value Error is encountered during finding the auc_score. Assign the AUC to 0 now.")
            acc_score = 0.0
            auc_score = 0.0
            auc_score_current = 0.0
            loss = 999999.9
            
	#confusion matrix of students
        if makefile:
            filename= '/content/drive/MyDrive/Project_re/results/confusion_information_'+ str(mode)+ '.csv'
            tp, tn, fp, fn = 0, 0, 0, 0
            f = open(filename, 'w')
            for i in range(len(total_confusion_matrix)):
                data = ' ,' + ','.join(total_confusion_matrix[i]) + '\n' 
                tp += total_confusion_matrix[i].count('TP')
                tn += total_confusion_matrix[i].count('TN')
                fp += total_confusion_matrix[i].count('FP')
                fn += total_confusion_matrix[i].count('FN')
                f.write(data)
            f.close()
        
        print('ACC :{}, AUC:{}, AUC_SCORE_CURRENT:{}'.format(acc_score, auc_score, auc_score_current))

        return acc_score, auc_score, auc_score_current, loss, total_confusion_matrix

    def run_optimization(self):
        num_epochs = self.num_epochs
        num_runs = self.num_runs
        sess = self.sess

        total_auc = 0.0
        self.accs = []
        self.aucs = []
        self.test_accs = []
        self.test_aucs = []
        self.test_aucs_current = []
        self.aucs_current = []
        self.wavinesses_l1 = []
        self.wavinesses_l2 = []
        self.consistency_m1 = []
        self.consistency_m2 = []
        for run_idx in range(num_runs):
            print("\n{0} th repeat training ..".format(run_idx+1))
            self.run_count = run_idx
            sess.run(tf.global_variables_initializer())
            acc_test = 0.0
            auc_test = 0.0
            auc_current_test = 0.0
            best_valid_acc = 0.0
            best_valid_auc = 0.0
            best_valid_auc_current = 0.0 # the auc_current when the test_auc is the best.
            best_waviness_l1 = 0.0
            best_waviness_l2 = 0.0
            best_consistency_m1 = 0.0
            best_consistency_m2 = 0.0

            best_epoch_idx = 0
            for epoch_idx in range(num_epochs):
                epoch_start_time = time.time()
                local_tz = pytz.timezone('Asia/Seoul')
                date = datetime.datetime.now(datetime.timezone.utc)
                current_time = date.astimezone(local_tz).strftime('%Y-%m-%d %H:%M:%S %Z')
                acc_train, auc_train, loss_train = self.train()
                self._log('Current time : {}'.format(current_time))

                self._log(
                    'Epoch {0:>4}, Train ACC: {1:.5}, Train AUC: {2:.5}, Train Loss: {3:.5}'.format(epoch_idx + 1, acc_train, auc_train, loss_train))

                acc_valid, auc_valid, auc_current_valid, loss_valid, _ = self.evaluate('valid')
                valid_msg = "Epoch {:>4}, Valid ACC: {:.5}, Valid AUC: {:.5}, Valid AUC Curr: {:.5}, Valid Loss: {:.5}".format(
                    epoch_idx + 1,
                    acc_valid,
                    auc_valid,
                    auc_current_valid,
                    loss_valid)

                if auc_train == 0 and auc_valid == 0:
                    self._log("ValueError occur, break the epoch loop.")
                    break

                if acc_valid > best_valid_acc:
                    valid_msg += "*"
                    best_epoch_idx = epoch_idx
                    best_valid_acc = acc_valid
                    best_valid_auc = auc_valid
                    best_valid_auc_current = auc_current_valid
                    best_waviness_l1, best_waviness_l2 = self.waviness('valid')

                    acc_test, auc_test, auc_current_test, loss_test, _ = self.evaluate('test', makefile=True)
                    valid_msg += "\nEpoch {:>4}, Test ACC: {:.5}, Test AUC: {:.5}, Test AUC Curr: {:.5}, Test Loss: {:.5}".format(
                        epoch_idx + 1,
                        acc_test,
                        auc_test,
                        auc_current_test,
                        loss_test)

                    m1, m2 = self.consistency('valid')
                    best_consistency_m1 = m1
                    best_consistency_m2 = m2

                    valid_msg += "\nw_l1: {0:5}, w_l2: {1:5}".format(best_waviness_l1, best_waviness_l2)
                    valid_msg += "\nm1: {0:5}, m2: {1:5}".format(best_consistency_m1, best_consistency_m2)
                    if self.save:
                        valid_msg += ". Saving the model"
                        self.save_model()
                    
                self._log(valid_msg)

                epoch_end_time = time.time()
                self._log("time used for this epoch: {0}s".format(epoch_end_time - epoch_start_time))
                self._log(SPLIT_MSG)

                if epoch_idx - best_epoch_idx >= 10:
                    self._log("No improvement shown in 10 epochs. Quit Training.")
                    break
                sys.stdout.flush()
                self.data_train.shuffle()
                
            self._log("The best validation result occured at: {0}-th epoch, with validation ACC: {1:.5} and AUC: {2:.5}".format(
                best_epoch_idx + 1, best_valid_acc, best_valid_auc))
            self._log("The best testing result occured at: {0}-th epoch, with testing ACC: {1:.5} and AUC: {2:.5}".format(
                best_epoch_idx + 1, acc_test, auc_test))
            
            self._log(SPLIT_MSG * 3)
            self.wavinesses_l1.append(best_waviness_l1)
            self.wavinesses_l2.append(best_waviness_l2)
            self.accs.append(best_valid_acc)
            self.aucs.append(best_valid_auc)
            self.test_accs.append(acc_test)
            self.test_aucs.append(auc_test)
            self.aucs_current.append(best_valid_auc_current)
            self.test_aucs_current.append(auc_current_test)
            self.consistency_m1.append(best_consistency_m1)
            self.consistency_m2.append(best_consistency_m2)
        avg_acc = np.average(self.accs)
        avg_auc = np.average(self.aucs)
        avg_test_acc = np.average(self.test_accs)
        avg_test_auc = np.average(self.test_aucs)
        avg_auc_current = np.average(self.aucs_current)
        avg_test_auc_current = np.average(self.test_aucs_current)
        avg_waviness_l1 = np.average(self.wavinesses_l1)
        avg_waviness_l2 = np.average(self.wavinesses_l2)
        avg_consistency_m1 = np.average(self.consistency_m1)
        avg_consistency_m2 = np.average(self.consistency_m2)

        self._log("average validation ACC for {0} runs: {1}".format(num_runs, avg_acc))
        self._log("average validation AUC for {0} runs: {1}".format(num_runs, avg_auc))
        self._log("average validation AUC Current for {0} runs: {1}".format(num_runs, avg_auc_current))
        self._log("\naverage waviness-l1 for {0} runs: {1}".format(num_runs, avg_waviness_l1))
        self._log("average waviness-l2 for {0} runs: {1}".format(num_runs, avg_waviness_l2))
        self._log("average consistency_m1 for {0} runs: {1}".format(num_runs, avg_consistency_m1))
        self._log("average consistency_m1 for {0} runs: {1}".format(num_runs, avg_consistency_m2))
        
        self._log("\ntest ACC for {0} runs : {1}".format(num_runs, self.test_accs))
        self._log("test AUC for {0} runs : {1}".format(num_runs, self.test_aucs))
        self._log("\naverage test ACC for {0} runs: {1}".format(num_runs, avg_test_acc))
        self._log("average test AUC for {0} runs: {1}".format(num_runs, avg_test_auc))
        self._log("average test AUC Current for {0} runs: {1}\n".format(num_runs, avg_test_auc_current))
        
        self._log("latex: \n" + self.auc_summary_in_latex())
        return avg_test_acc

    def save_model(self):
        save_dir = self.ckpt_save_dir
        sess = self.sess
        saver = tf.train.Saver()
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        save_path = os.path.join(save_dir, self.model_name)
        pb_save_path = os.path.join(save_dir, 'model.pb')
        pb_txt_save_path = os.path.join(save_dir, 'model.pbtxt')
        saver.save(sess=sess, save_path=save_path)
        tf.io.write_graph(sess.graph_def, '.', pb_save_path, as_text=False)
        tf.io.write_graph(sess.graph_def, '.', pb_txt_save_path, as_text=True)

    def load_model(self):
        save_dir = os.path.join(self.ckpt_save_dir, 'run_{}'.format(self.run_count), self.model_name)
        sess = self.sess
        saver = tf.train.Saver()
        save_path = os.path.join(save_dir, self.model_name)
        if os.path.exists(save_path):
            saver.restore(sess=sess, save_path=save_path)
        else:
            self._log("No model found at {}".format(save_path))

    def get_hidden_layer_output(self, problem_seqs, correct_seqs, layer):
        model = self.model
        sess = self.sess
        num_layer = len(model.hidden_layer_structure)
        assert layer < num_layer, "There are only {0} layers. indexed from 0.".format(num_layer)

        input_processor = OriginalInputProcessor()
        X, y_seq, y_corr = input_processor.process_problems_and_corrects(problem_seqs=problem_seqs,
                                                                         correct_seqs=correct_seqs,
                                                                         num_problems=self.num_problems)

        feed_dict = {
            model.X: X,
            model.y_seq: y_seq,
            model.y_corr: y_corr,
            model.keep_prob: 1.0,
        }

        hidden_layers_outputs = sess.run(
            model.hidden_layers_outputs,
            feed_dict=feed_dict
        )

        result = hidden_layers_outputs[layer]
        return result

    def get_output_layer(self, problem_seqs, correct_seqs):
        model = self.model
        sess = self.sess

        input_processor = OriginalInputProcessor()
        X, y_seq, y_corr = input_processor.process_problems_and_corrects(problem_seqs=problem_seqs,
                                                                         correct_seqs=correct_seqs,
                                                                         num_problems=self.num_problems,
                                                                         is_train=False)

        feed_dict = {
            model.X: X,
            model.y_seq: y_seq,
            model.y_corr: y_corr,
            model.keep_prob: 1.0,
        }

        pred_seqs = sess.run(
            model.preds,
            feed_dict=feed_dict
        )

        return pred_seqs

    def _log(self, log_msg):
        print(log_msg)
        if self.logging:
            with open(self.log_file_path, "a+") as f:
                f.write(log_msg + '\n')

    def auc_summary_in_latex(self):
        # def mean_confidence_interval(data, confidence=0.95):
        #     import scipy.stats as st
        #     import numpy as np
        #     a = 1.0 * np.array(data)
        #     n = len(a)
        #     m, se = np.mean(a), st.sem(a)
        #     h = se * st.t.ppf((1 + confidence) / 2., n - 1)
        #     return m, h
        #
        # assert len(aucs) > 1, "There should be at least two auc scores to find the interval."
        cell_type_str = repr(self.network_config['rnn_cell']).split('.')[-1][:-6]
        num_layers_str = str(len(self.network_config['hidden_layer_structure']))
        layer_structure_str = ", ".join([str(i) for i in self.network_config['hidden_layer_structure']])

        # experiment result
        acc_mean = np.average(self.accs)
        acc_std = np.std(self.accs)
        
        auc_mean = np.average(self.aucs)
        auc_std = np.std(self.aucs)

        auc_current_mean = np.average(self.aucs_current)
        auc_current_std = np.std(self.aucs_current)

        waviness_l1_mean = np.average(self.wavinesses_l1)
        waviness_l1_std = np.std(self.wavinesses_l1)

        waviness_l2_mean = np.average(self.wavinesses_l2)
        waviness_l2_std = np.std(self.wavinesses_l2)

        consistency_m1_mean = np.average(self.consistency_m1)
        consistency_m1_std = np.std(self.consistency_m1)

        consistency_m2_mean = np.average(self.consistency_m2)
        consistency_m2_std = np.std(self.consistency_m2)

        # cell_type & num. layer & layer_structure & learning rate & keep prob & Avg. AUC & Avg. Waviness
        # LSTM & 1 & (200,) & 0.0100 & 0.500 & 0.010 & 0.82500 $\pm$ 0.000496\\
        result_cols = [
            'cell_type',
            'num. layer',
            'layer_structure',
            'learning rate',
            'keep prob.',
            '$\lambda_o$',
            '$\lambda_{w_1}$',
            '$\lambda_{w_2}$',
            'Avg. ACC(N)',
            'Avg. AUC(N)',
            'Avg. AUC(C)',
            'Avg. $w_1$',
            'Avg. $w_2$',
            'Avg. $m_1$',
            'Avg. $m_2$',
        ]

        result_data = [
            cell_type_str,
            num_layers_str,
            layer_structure_str,
            "{:.4f}".format(self.network_config['learning_rate']),
            "{:.4f}".format(self.network_config['keep_prob']),
            "{:.4f}".format(self.network_config['lambda_o']),
            "{:.4f}".format(self.network_config['lambda_w1']),
            "{:.4f}".format(self.network_config['lambda_w2']),
            "{} $\pm$ {}".format(acc_mean, acc_std),
            "{} $\pm$ {}".format(auc_mean, auc_std),
            "{} $\pm$ {}".format(auc_current_mean, auc_current_std),
            "{} $\pm$ {}".format(waviness_l1_mean, waviness_l1_std),
            "{} $\pm$ {}".format(waviness_l2_mean, waviness_l2_std),
            "{} $\pm$ {}".format(consistency_m1_mean, consistency_m1_std),
            "{} $\pm$ {}".format(consistency_m2_mean, consistency_m2_std),
        ]

        latex_str = " & ".join(result_cols)
        latex_str += "\\\\ \n"

        latex_str += " & ".join(result_data)
        latex_str += "\\\\ \n"
        return latex_str

    def plot_output_layer(self, problem_seq, correct_seq, target_problem_ids=None):
        import matplotlib.pyplot as plt
        import seaborn as sns
        problem_ids_answered = sorted(set(problem_seq))
        if target_problem_ids is None:
            target_problem_ids = problem_ids_answered

        # get_output_layer return output in shape (1, 38, 124)
        output = self.get_output_layer(problem_seqs=[problem_seq], correct_seqs=[correct_seq])[0]  # shape (38, 124)
        output = output[:, target_problem_ids]  # shape (38, ?)
        output = np.transpose(output)  # shape (?, 38)

        y_labels = target_problem_ids
        x_labels = ["({},{})".format(p, c) for p, c in zip(problem_seq, correct_seq)]
        df = pd.DataFrame(output)
        df.columns = x_labels
        df.index = y_labels

        return sns.heatmap(df, vmin=0, vmax=1, cmap=plt.cm.Blues)

    def plot_hidden_layer(self, problem_seq, correct_seq, layer):
        import matplotlib.pyplot as plt
        import seaborn as sns
        output = self.get_hidden_layer_output(problem_seqs=[problem_seq], correct_seqs=[correct_seq], layer=layer)
        output = output[0]  # ignore the batch_idx
        output = np.transpose(output)

        y_labels = range(output.shape[0])
        x_labels = ["({},{})".format(p, c) for p, c in zip(problem_seq, correct_seq)]
        df = pd.DataFrame(output)
        df.columns = x_labels
        df.index = y_labels

        return sns.heatmap(df, cmap='RdBu')

    def waviness(self, mode='valid'):
        if mode == 'train':
            data = self.data_train
            is_train=True
        elif mode == 'valid':
            data = self.data_valid
            is_train=False
        else:
            data = self.data_test
            is_train=False
        data.reset_cursor()
        model = self.model
        sess = self.sess

        waviness_l1 = 0.0
        waviness_l2 = 0.0
        total_num_steps = 0.0
        for batch_idx in range(data.num_batches):
            # print('batch:', batch_idx, end='\r')
            X_batch, y_seq_batch, y_corr_batch = data.next_batch(is_train)
            feed_dict = {
                model.X: X_batch,
                model.y_seq: y_seq_batch,
                model.y_corr: y_corr_batch,
                model.keep_prob: 1,
            }
            _waviness_l1, _waviness_l2, _total_num_steps = sess.run(
                [model.waviness_l1,
                 model.waviness_l2,
                 model.total_num_steps],
                feed_dict=feed_dict
            )
            waviness_l1 += _waviness_l1 * _total_num_steps
            waviness_l2 += _waviness_l2 * _total_num_steps
            total_num_steps += _total_num_steps
        waviness_l1 /= total_num_steps
        waviness_l2 /= total_num_steps
        waviness_l2 = np.sqrt(waviness_l2)

        return waviness_l1, waviness_l2


    def waviness_np(self, mode='valid'):
        if mode == 'train':
            data = self.data_train
            is_train=True
        elif mode == 'valid':
            data = self.data_valid
            is_train=False
        else:
            data = self.data_test
            is_train=False
        data.reset_cursor()
        model = self.model
        sess = self.sess

        waviness_l1 = 0.0
        waviness_l2 = 0.0
        total_num_steps = 0.0
        for batch_idx in range(data.num_batches):
            X_batch, y_seq_batch, y_corr_batch = data.next_batch(is_train)

            feed_dict = {
                model.X: X_batch,
                model.y_seq: y_seq_batch,
                model.y_corr: y_corr_batch,
                model.keep_prob: 1,
            }
            pred_seqs = sess.run(
                model.preds,
                feed_dict=feed_dict
            )

            # finding w1, w2 for this batch
            w1 = np.sum(np.abs(pred_seqs[:, 1:, :] - pred_seqs[:, :-1, :]))
            w2 = np.sum(np.square(pred_seqs[:, 1:, :] - pred_seqs[:, :-1, :]))

            seq_length_batch = np.sum(_seq_length(y_seq_batch[:, 1:, :]))
            waviness_l1 += w1
            waviness_l2 += w2
            total_num_steps += seq_length_batch

            # print('batch:{}, w1:{}, w2:{}, length:{}'.format(batch_idx, w1, w2, seq_length_batch), end='\r')

        waviness_l1 /= (total_num_steps * data.num_problems)
        waviness_l2 /= (total_num_steps * data.num_problems)
        waviness_l2 = np.sqrt(waviness_l2)

        return waviness_l1, waviness_l2

    def _reconstruction_accurarcy(self, mode='valid'):
        if mode == 'train':
            data = self.data_train
        elif mode == 'valid':
            data = self.data_valid
        else:
            data = self.data_test
        data.reset_cursor()

        problem_seqs = data.problem_seqs
        correct_seqs = data.correct_seqs
        num_interactions = 0
        sign_diff_score = 0
        diff_score = 0
        for i in range(len(problem_seqs)):
            if i%20 == 0:
                print(i, end='\r')
            problem_seq = problem_seqs[i]
            correct_seq = correct_seqs[i]
            outputs = self.get_output_layer([problem_seq], [correct_seq]) # shape: (batch, time, num_problems)

            for j in range(1, len(problem_seq)): # exclude the prediction of the first output
                target_id = problem_seq[j]
                label = correct_seq[j]
                score = 1.0 if label==1 else -1.0

                prev_pred = outputs[0][j-1][target_id]
                curr_pred = outputs[0][j][target_id]
                pred_diff = curr_pred - prev_pred
                pred_sign_diff = np.sign(pred_diff)

                sign_diff_score += pred_sign_diff * score
                diff_score += pred_diff * score
                num_interactions += 1
        return (sign_diff_score, diff_score, num_interactions)

    def consistency(self, mode='valid'):
        if mode == 'train':
            data = self.data_train
            is_train=True
        elif mode == 'valid':
            data = self.data_valid
            is_train=False
        else:
            data = self.data_test
            is_train=False
        data.reset_cursor()
        model = self.model
        sess = self.sess

        consistency_m1 = 0.0
        consistency_m2 = 0.0
        total_num_steps = 0.0
        for batch_idx in range(data.num_batches):
            # X_batch: one hot encoded (q_t, a_t)
            # y_seq_batch: one hot encoded (q_t), \deltadm{q_t}
            # y_corr_batch: one hot encoded (a_t)
            X_batch, y_seq_batch, y_corr_batch = data.next_batch(is_train)
            seq_length_batch = np.sum(_seq_length(y_seq_batch[:, 1:, :]))

            feed_dict = {
                model.X: X_batch,
                model.y_seq: y_seq_batch,
                model.y_corr: y_corr_batch,
                model.keep_prob: 1,
            }
            pred_seqs = sess.run(
                model.preds,
                feed_dict=feed_dict
            )

            # finding m1, m2 for this batch
            base = y_seq_batch[:, 1:, :].copy()
            base[:] = -1.0
            coefficient = np.sum( (np.power(base, 1 - y_corr_batch[:, 1:, :])) * y_seq_batch[:, 1:, :], axis=2)

            m1 = np.sum(
                coefficient * np.sign(np.sum(
                    (pred_seqs[:, 1:, :] - pred_seqs[:, :-1, :]) * y_seq_batch[:, 1:, :], #y_t-y_{t-1} \dot
                    axis=2
                ))
            )
            m2 = np.sum(
                coefficient * np.sum(
                    (pred_seqs[:, 1:, :] - pred_seqs[:, :-1, :]) * y_seq_batch[:, 1:, :],
                    axis=2
                )
            )

            consistency_m1 += m1
            consistency_m2 += m2
            total_num_steps += seq_length_batch

        consistency_m1 /= (total_num_steps)
        consistency_m2 /= (total_num_steps)

        return consistency_m1, consistency_m2

##모델 학습

In [ ]:
import platform, psutil 

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# 이번 프로젝트에서는 LSTM을 활용하였지만, 추후에 다른 모델로 학습 시도 가능 
rnn_cells = {
    "LSTM": tf.contrib.rnn.LSTMCell,
    "GRU": tf.contrib.rnn.GRUCell,
    "BasicRNN": tf.contrib.rnn.BasicRNNCell,
    "LayerNormBasicLSTM": tf.contrib.rnn.LayerNormBasicLSTMCell,
}

num_runs = 1
num_epochs = 25
batch_size = 32
keep_prob = 0.8656542586183774

network_config = {}
network_config['batch_size'] = batch_size
network_config['hidden_layer_structure'] = [102, ]
network_config['learning_rate'] = 0.004155923499457689
network_config['keep_prob'] = keep_prob
network_config['rnn_cell'] = rnn_cells['LSTM']
network_config['max_grad_norm'] = 5.0
network_config['lambda_w1'] = 0.03
network_config['lambda_w2'] = 3.0
network_config['lambda_o'] = 0.1

train_path = '/content/drive/MyDrive/Project_re/data/i-scream_train.csv'
valid_path = '/content/drive/MyDrive/Project_re/data/i-scream_valid.csv'
test_path = '/content/drive/MyDrive/Project_re/data/i-scream_test.csv'
save_dir_prefix = '/content/drive/MyDrive/Project_re/results/'

In [ ]:
def main():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    
    #CPU, GPU, RAM, HDD, OS
    local_tz = pytz.timezone('Asia/Seoul')
    date = datetime.datetime.now(datetime.timezone.utc)
    current_time = date.astimezone(local_tz).strftime('%Y-%m-%d %H:%M:%S %Z')
    print('Current time : {}, TensorFlow version :{}'.format(current_time, tf.__version__))
    print('Current time : {},  OS infromation :{}'.format(current_time, platform.system()))
    print('Current time : {}, OS version :{}'.format(current_time, platform.version()))
    print('Current time : {},  Process information :{}'.format(current_time, platform.processor()))
    print('Current time : {},  CPU_count :{}'.format(current_time, os.cpu_count()))
    print('Current time : {},  RAM_size :{}'.format(current_time, str(round(psutil.virtual_memory().total/(1024.0**3)))+"(GB)"))
    print(os.system('nvidia-smi'))

    data = DKTData(train_path, valid_path, test_path, batch_size=batch_size)
    data_train = data.train
    data_valid = data.valid
    data_test = data.test
    num_problems = data.num_problems

    dkt = DKT(sess, data_train, data_valid, data_test, num_problems, network_config,
              save_dir_prefix=save_dir_prefix,
              num_runs=num_runs, num_epochs=num_epochs,
              keep_prob=keep_prob, logging=True, save=True)

    # run optimization of the created model
    dkt.model.build_graph()
    dkt.run_optimization()

    # close the session
    sess.close()

In [ ]:
start_time = time.time()
main()
end_time = time.time()
print("program run for: {0}s".format(end_time - start_time))

Current time : 2021-10-18 21:11:32 KST, TensorFlow version :1.15.2
Current time : 2021-10-18 21:11:32 KST,  OS infromation :Linux
Current time : 2021-10-18 21:11:32 KST, OS version :#1 SMP Sat Jun 5 09:50:34 PDT 2021
Current time : 2021-10-18 21:11:32 KST,  Process information :x86_64
Current time : 2021-10-18 21:11:32 KST,  CPU_count :2
Current time : 2021-10-18 21:11:32 KST,  RAM_size :13(GB)
0
Reading /content/drive/MyDrive/Project_re/data/i-scream_train.csv
44634 lines was read
max_num_problems_answered: 1200
num_problems: 1865
The number of data is 14878
Finish reading data.
Reading /content/drive/MyDrive/Project_re/data/i-scream_valid.csv
8937 lines was read
max_num_problems_answered: 1173
num_problems: 1865
The number of data is 2979
Finish reading data.
Reading /content/drive/MyDrive/Project_re/data/i-scream_test.csv
13404 lines was read
max_num_problems_answered: 1241
num_problems: 1865
The number of data is 4468
Finish reading data.
Network Configuration:
batch_size: 32
hidde

train batch...: 100%|██████████| 465/465 [12:11<00:00,  1.57s/it]


Current time : 2021-10-18 21:11:35 KST
Epoch    1, Train ACC: 0.69067, Train AUC: 0.71218, Train Loss: 0.65591


valid batch...: 100%|██████████| 94/94 [01:42<00:00,  1.09s/it]


ACC :0.7567794538167979, AUC:0.7801769393644569, AUC_SCORE_CURRENT:0.9060543664505094


...

ACC :0.797567794538168, AUC:0.8304889387137895, AUC_SCORE_CURRENT:0.919762132897324
Epoch   16, Valid ACC: 0.79757, Valid AUC: 0.83049, Valid AUC Curr: 0.91976, Valid Loss: 0.50246
time used for this epoch: 894.9953649044037s
***********


train batch...: 100%|██████████| 465/465 [13:17<00:00,  1.72s/it]


Current time : 2021-10-19 01:56:50 KST
Epoch   17, Train ACC: 0.7954, Train AUC: 0.86189, Train Loss: 0.48655


valid batch...: 100%|██████████| 94/94 [01:48<00:00,  1.16s/it]


ACC :0.7973844224043541, AUC:0.8301150567156432, AUC_SCORE_CURRENT:0.9192036221332892
Epoch   17, Valid ACC: 0.79738, Valid AUC: 0.83012, Valid AUC Curr: 0.9192, Valid Loss: 0.50314
time used for this epoch: 909.237779378891s
***********
No improvement shown in 10 epochs. Quit Training.
The best validation result occured at: 7-th epoch, with validation ACC: 0.80112 and AUC: 0.83111
The best testing result occured at: 7-th epoch, with testing ACC: 0.79958 and AUC: 0.83272
*********************************
average validation ACC for 1 runs: 0.8011236193938621
average validation AUC for 1 runs: 0.831106785027208
average validation AUC Current for 1 runs: 0.9372570241223359

average waviness-l1 for 1 runs: 0.01659867754862962
average waviness-l2 for 1 runs: 0.03410982171254454
average consistency_m1 for 1 runs: 0.33628323287729484
average consistency_m1 for 1 runs: 0.04723263632723264

test ACC for 1 runs : [0.7995843657021101]
test AUC for 1 runs : [0.8327157808747274]

average test ACC f

##test
코드가 잘 돌아가는지 1 epoch만 테스트 수행 

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
rnn_cells = {
    "LSTM": tf.contrib.rnn.LSTMCell,
    "GRU": tf.contrib.rnn.GRUCell,
    "BasicRNN": tf.contrib.rnn.BasicRNNCell,
    "LayerNormBasicLSTM": tf.contrib.rnn.LayerNormBasicLSTMCell,
}

num_runs = 1
num_epochs = 1
batch_size = 1
keep_prob = 0.8656542586183774

network_config = {}
network_config['batch_size'] = batch_size
network_config['hidden_layer_structure'] = [102, ]
network_config['learning_rate'] = 0.004155923499457689
network_config['keep_prob'] = keep_prob
network_config['rnn_cell'] = rnn_cells['LSTM']
network_config['max_grad_norm'] = 5.0
network_config['lambda_w1'] = 0.03
network_config['lambda_w2'] = 3.0
network_config['lambda_o'] = 0.1

train_path = '/content/drive/MyDrive/Project_re/data/i-scream_train.csv'
valid_path = '/content/drive/MyDrive/Project_re/data/i-scream_valid.csv'
test_path = '/content/drive/MyDrive/Project_re/data/i-scream_test.csv'
save_dir_prefix = '/content/drive/MyDrive/Project_re/test/results/'

In [ ]:
def confusion_matrix(filename):
    tp, tn, fp, fn = 0, 0, 0, 0 
    rows = []
    with open(filename, 'r') as f: 
        print("Reading {0}".format(filename))
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            rows.append(row)
        print("{0} lines was read".format(len(rows)))

    for i in range(len(rows)):
        tp += rows[i].count('TP')
        tn += rows[i].count('TN')
        fp += rows[i].count('FP')
        fn += rows[i].count('FN')
    confusion_matrix = [tp, tn, fp, fn]
    print(confusion_matrix)
    return confusion_matrix

In [ ]:
def write_csv(filename1, filename2, confusion_matrix): 
    # read the csv file
    rows1, rows2 = [], []
    with open(filename1, 'r') as f: 
        print("Reading {0}".format(filename1))
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            rows1.append(row)
        print("{0} lines was read".format(len(rows1)))

    with open(filename2, 'r') as f: 
        print("Reading {0}".format(filename2))
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            rows2.append(row)
        print("{0} lines was read".format(len(rows2)))

    with open('/content/drive/MyDrive/Project_re/results/test_results.csv', 'w', newline="") as f: 
        writer = csv.writer(f)
        for i in range(0, len(rows1), 3):
            writer.writerow(rows1[i])
            writer.writerow(rows1[i+1])
            writer.writerow(rows1[i+2])
            writer.writerow(rows2[i//3])
        writer.writerow(['TP : {}'.format(confusion_matrix[0])])
        writer.writerow(['TN : {}'.format(confusion_matrix[1])])
        writer.writerow(['FP : {}'.format(confusion_matrix[2])])
        writer.writerow(['FN : {}'.format(confusion_matrix[3])])

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

data = DKTData(train_path, valid_path, test_path, batch_size=batch_size)
data_train = data.train
data_valid = data.valid
data_test = data.test
num_problems = data.num_problems

dkt = DKT(sess, data_train, data_valid, data_test, num_problems, network_config,
            save_dir_prefix=save_dir_prefix,
            num_runs=num_runs, num_epochs=num_epochs,
            keep_prob=keep_prob, logging=True, save=True)

dkt.model.build_graph()
#saver = tf.train.Saver()
#saver.restore(sess=sess, save_path=best_model_save_path)
dkt.run_optimization()
sess.close()

Reading /content/drive/MyDrive/Project_re/data/i-scream_train.csv
44634 lines was read
max_num_problems_answered: 1200
num_problems: 1865
The number of data is 14878
Finish reading data.
Reading /content/drive/MyDrive/Project_re/data/i-scream_valid.csv
8937 lines was read
max_num_problems_answered: 1173
num_problems: 1865
The number of data is 2979
Finish reading data.
Reading /content/drive/MyDrive/Project_re/data/i-scream_test.csv
13404 lines was read
max_num_problems_answered: 1241
num_problems: 1865
The number of data is 4468
Finish reading data.
Network Configuration:
batch_size: 1
hidden_layer_structure: [102]
learning_rate: 0.004155923499457689
keep_prob: 0.8656542586183774
rnn_cell: <class 'tensorflow.python.ops.rnn_cell_impl.LSTMCell'>
max_grad_norm: 5.0
lambda_w1: 0.03
lambda_w2: 3.0
lambda_o: 0.1
Num of problems: 1865
Num of run: 1
Max num of run: 1
Keep Prob: 0.8656542586183774
Creating placeholder...
Creating Loss...
LSTM input shape: (?, ?, 3730)
Creating Loss...
Create o

train batch...: 100%|██████████| 14878/14878 [48:28<00:00,  5.12it/s]


Current time : 2021-10-20 15:17:30 KST
Epoch    1, Train ACC: 0.71962, Train AUC: 0.75194, Train Loss: 0.63712


valid batch...: 100%|██████████| 2979/2979 [06:09<00:00,  8.07it/s]


ACC :0.7280192620467519, AUC:0.758555408029559, AUC_SCORE_CURRENT:0.8806657182712045


test batch...: 100%|██████████| 4468/4468 [09:19<00:00,  7.99it/s]


ACC :0.7263489716029956, AUC:0.7597076863579955, AUC_SCORE_CURRENT:0.88038611758899
Epoch    1, Valid ACC: 0.72802, Valid AUC: 0.75856, Valid AUC Curr: 0.88067, Valid Loss: 0.63181*
Epoch    1, Test ACC: 0.72635, Test AUC: 0.75971, Test AUC Curr: 0.88039, Test Loss: 0.63104
w_l1: 0.00941034362503607, w_l2: 0.021765419623402886
m1: 0.24833902052704873, m2: 0.025368519435067426. Saving the model
time used for this epoch: 4554.812645196915s
***********
The best validation result occured at: 1-th epoch, with validation ACC: 0.72802 and AUC: 0.75856
The best testing result occured at: 1-th epoch, with testing ACC: 0.72635 and AUC: 0.75971
*********************************
average validation ACC for 1 runs: 0.7280192620467519
average validation AUC for 1 runs: 0.758555408029559
average validation AUC Current for 1 runs: 0.8806657182712045

average waviness-l1 for 1 runs: 0.00941034362503607
average waviness-l2 for 1 runs: 0.021765419623402886
average consistency_m1 for 1 runs: 0.248339020527

In [ ]:
info = confusion_matrix('/content/drive/MyDrive/Project_re/results/confusion_information_test.csv')
write_csv('/content/drive/MyDrive/Project_re/data/i-scream_test.csv', '/content/drive/MyDrive/Project_re/results/confusion_information_test.csv', info)
os.remove('/content/drive/MyDrive/Project_re/results/confusion_information_test.csv')

Reading /content/drive/MyDrive/Project_re/results/confusion_information_test.csv
4468 lines was read
[303597, 110926, 72911, 83260]
Reading /content/drive/MyDrive/Project_re/data/i-scream_test.csv
13404 lines was read
Reading /content/drive/MyDrive/Project_re/results/confusion_information_test.csv
4468 lines was read


In [ ]:
# confusion_matrix = [tp, tn, fp, fn]

print('confusion_matrix :', info)
print('baseline :', (info[0] + info[3]) / sum(info))

confusion_matrix : [303597, 110926, 72911, 83260]
baseline : 0.6778711533676541
